In [1]:
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Line
import akshare as ak
macro_china_shrzgm_df = ak.macro_china_shrzgm()
print(macro_china_shrzgm_df)
macro_china_shrzgm_df['年度'] = macro_china_shrzgm_df['月份'].str[:4]


        月份  社会融资规模增量  其中-人民币贷款  其中-委托贷款外币贷款  其中-委托贷款  其中-信托贷款  其中-未贴现银行承兑汇票  \
0   201501     20516     14708          212      832       52          1946   
1   201502     13609     11437         -146     1299       38          -592   
2   201503     12433      9920           -4     1111      -77          -910   
3   201504     10582      8045         -265      344      -46           -74   
4   201505     12397      8510           81      324     -195           961   
..     ...       ...       ...          ...      ...      ...           ...   
91  202208     24712     13344         -826     1755     -472          3486   
92  202209     35411     25686         -713     1508     -191           132   
93  202210      9134      4431         -724      470      -61         -2156   
94  202211     19837     11448         -648      -88     -365           191   
95  202212     13058     14401        -1665     -101     -764          -554   

    其中-企业债券  其中-非金融企业境内股票融资  
0      1868          

In [72]:
x_data=list(range(1,13))
shrzgm = (
    Line(init_opts=opts.InitOpts(
        width='1200px', 
        height='600px', 
        )
    )
    .add_xaxis(xaxis_data=x_data)
)

for y in macro_china_shrzgm_df.groupby('年度'):
    shrzgm.add_yaxis(y[0],y[1]['社会融资规模增量'].to_list())

shrzgm.set_global_opts(
    title_opts=opts.TitleOpts(title="历年社会融资规模增量"),
    legend_opts=opts.LegendOpts(is_show=True, textstyle_opts=opts.TextStyleOpts(color='white')),
    tooltip_opts=opts.TooltipOpts(
        is_show=True,
        trigger_on="mousemove|click",
        axis_pointer_type='cross'),
    xaxis_opts=opts.AxisOpts(
        type_="category",
        splitline_opts=opts.SplitLineOpts(is_show=False, linestyle_opts=opts.LineStyleOpts(type_='dotted'))
    ),
    yaxis_opts=opts.AxisOpts(
        type_="value",
        splitline_opts=opts.SplitLineOpts(is_show=False, linestyle_opts=opts.LineStyleOpts(type_='dotted')),
    ),
)
shrzgm.set_series_opts(label_opts=opts.LabelOpts(is_show=False))

In [73]:
shrzgm.render_notebook()

In [71]:
print(x_data)
for i in macro_china_shrzgm_df.groupby('年度'):
    print(i[1]['社会融资规模增量'].to_list())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[20516, 13609, 12433, 10582, 12397, 18384, 7511, 11097, 13571, 5593, 10255, 18114]
[34758, 8312, 23931, 7809, 6770, 16479, 4791, 14605, 17115, 8865, 18328, 16397]
[36970, 10908, 21186, 13880, 10631, 17718, 11956, 14895, 18335, 10357, 16128, 11481]
[30793, 11894, 15848, 17761, 9518, 14877, 12254, 19391, 21691, 7420, 15239, 15898]
[46791, 9665, 29602, 16710, 17124, 26243, 12872, 21956, 25142, 8680, 19937, 22013]
[50535, 8737, 51838, 31027, 31866, 34681, 16928, 35853, 34693, 13929, 21355, 17192]
[51884, 17243, 33762, 18570, 19522, 37017, 10752, 29893, 29026, 16176, 25983, 23682]
[61759, 12170, 46565, 9327, 28415, 51926, 7785, 24712, 35411, 9134, 19837, 13058]


In [ ]:
x_data= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
y_data= [20516, 13609, 12433, 10582, 12397, 18384, 7511, 11097, 13571, 5593, 10255, 18114]
shrzgm = (
    Line(init_opts=opts.InitOpts(
        width='1200px', 
        height='600px', 
        )
    )
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis('1',y_data)
    .render_notebook()
)
